# Installing neccessary packages


# Install your packages here

In [1]:
install.packages('jsonlite')
install.packages('rvest')
install.packages('tidyverse')
install.packages('lubridate')
install.packages('httr')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("jsonlite"):
“installation of package ‘jsonlite’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("httr"):
“installation of package ‘httr’ had non-zero exit status”


# Load the libraries here

In [2]:
library(jsonlite)
library(httr)
library(rvest)
library(tidyverse)
library(lubridate)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()         masks stats::filter()
✖ purrr::flatten()        masks jsonlite::flatten()
✖ readr::guess_encoding() masks rvest::guess_encoding()
✖ dplyr::lag()            masks stats::lag()

Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




# trimStringArray function

In [3]:
trimStringArray = function(array) {
  if (is.array(array)) {
    for (i in 1:length(array())) {
      array[i] = trimws(array[i])
    }
  } else {
    array = trimws(array)
  }
  array
}

# ternaryOperatorFunction | Vectorized function

In [4]:
ternary = function(expression_1, expression_2, condition) {
    if(condition) {
        expression_1
    } else {
        expression_2
    }
}

ternary_v = Vectorize(ternary)

# Create a vectorized version of strsplit()

In [5]:
strsplit_custom = function (string, splitter, index) {
    strsplit(string, splitter)[[1]][index]
}

strsplit_custom_v = Vectorize(strsplit_custom)

# Scrape Teilnehmer von WM

In [6]:
scrapeColumnsWM = function(game_name) {

  # base url

  url = 'https://www.transfermarkt.de/weltmeisterschaft-2022/teilnehmer/pokalwettbewerb/WM22/saison_id/2021'

  # read url

  url %>%
    read_html() -> html

  # variables

  columnValues = list() # a list containing all values
  columnNames = NULL
  box = NULL # searching for the table_header
  slugs = c()
  club_ids = c()

  # Obtain the correct responsive-table by name of table_header

  html %>%
    html_elements('.box') -> boxes

  # Print the table headers

  boxes

  # Iterate over each table header to find the match with table_header
  # argument

  for (item in boxes) {

    #print(item)

    item %>%
      html_elements('.table-header') %>%
      html_text() %>%
      trimStringArray() -> name

    if (length(name) != 0 && grepl(tolower(game_name), tolower(name))) {
      box = item
      break
    }
  }

  box

  if(!is.null(box)) {
      # Obtain the responsive_table

      box %>%
          html_elements('.responsive-table') -> responsive_table

      # print the responsive-table

      responsive_table

      # scrape the url with the slugs and club ids of the countries

      responsive_table %>%
          html_elements('tr td:first-of-type a') %>%
          html_attr('href') %>%
          strsplit('/') -> raw_url_data

      raw_url_data

      # obtain the slug of all countries

      raw_url_data[1][[1]][2]
      raw_url_data[1][[1]][length(raw_url_data[1][[1]])]

      for (i in 1:length(raw_url_data)) {
          slugs = c(slugs, raw_url_data[i][[1]][2])
          club_ids = c(club_ids, raw_url_data[i][[1]][length(raw_url_data[1][[1]])])
      }

      # print slugs

      slugs

      # print club_ids

      club_ids

      # obtain the columnNames

      responsive_table %>%
          html_elements('th') %>%
          html_text -> columnNames

      # print columnNames

      columnNames

      # add slug and club_id to columnNames

      columnNames = c(columnNames, c('slug', 'club_id'))

      # print columnNames again

      columnNames

      # obtain length of a all true columns

      responsive_table %>%
          html_elements('tbody tr:nth-of-type(1) td') %>%
          length() -> trueColumnsLength

      # print the length of all true columns

      trueColumnsLength

      for (i in 1:trueColumnsLength) {
          # create css selector string
          # td:not([class*="no-border-rechts"]
          string = paste0('tbody tr td:not([class*="no-border-rechts"]):nth-of-type(', i,')')

          # print the css selector string

          string

          # obtain all row elements of specified column

          responsive_table %>%
              html_elements(string) -> html_node

          # print all row elements of specified column

          html_node

          # only add the values if they are valid!!!

          if(length(html_node) != 0) {
              # obtain the children (if any) from the row elements
              # of specified column

              html_node %>%
                  html_children() -> children

              # print the children

              children

              # check the number of children contained within

              num_of_children = length(children)

              # print the number of children (if any)

              num_of_children

              # check if a child contains any subchildren

              children %>%
                  html_children() -> subchildren

              # print subchildren

              subchildren

              # check the number of children contained within

              num_of_subchildren = length(subchildren)

              # print number of subchildren

              num_of_subchildren

              # dynammically apply a search for the values
              # according to the number of children

              if (num_of_subchildren > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a > span') %>%
                      html_text() -> values
              } else if (num_of_children > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a') %>%
                      html_attr('title') -> values
              } else {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_text -> values
              }

              # wrap the obtained values within a list

              list(values)

              # iterate over all children by

              values = trimStringArray(values)

              # Add this column to the columnValues list

              columnValues = append(columnValues, list(values))
          }
          else {
              print(paste0("Column ", i, " is not a valid value"))
          }

      }

      # add club_id and slug to values

      columnValues = append(columnValues, list(slugs))
      columnValues = append(columnValues, list(club_ids))

      # print columnValues

      columnValues

      # return list
      list(columnNames, columnValues)
  } else {
      # return NULL
      NULL
  }
}

# create Dataframe from scrapeColumnsWM

In [7]:
create_dataframeWM = function(name) {
  df_wm = scrapeColumnsWM(name)

  if(!(is.null(df_wm))) {
      columns = df_wm[[2]]
      columnNames = df_wm[[1]]

      #print(columnNames)

      df_wm = data.frame(columns[[1]])

      for (i in 2:length(columns)) {
          df_wm[i] = columns[[i]]
      }

      names(df_wm) = columnNames[2:length(columnNames)]
      df_wm
  } else {
      NULL
  }
}

In [8]:
df_wm = create_dataframeWM('teilnehmende teams an der wm')
df_wm

[1] "Column 1 is not a valid value"


Verein,Kader,&oslash-Alter,WM-Teilnahmen,Legionäre,Marktwert,&oslash-Marktwert,slug,club_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
England,27,"25,6",16,"11,1 %","1,29 Mrd. €","47,89 Mio. €",england,3299
Frankreich,25,"26,9",16,"76,0 %","1,06 Mrd. €","42,40 Mio. €",frankreich,3377
Brasilien,26,"27,2",22,"88,5 %","963,00 Mio. €","37,04 Mio. €",brasilien,3439
Deutschland,25,"26,6",20,"28,0 %","824,50 Mio. €","32,98 Mio. €",deutschland,3262
Portugal,24,"27,3",8,"87,5 %","772,00 Mio. €","32,17 Mio. €",portugal,3300
Spanien,24,"25,9",16,"29,2 %","751,00 Mio. €","31,29 Mio. €",spanien,3375
Argentinien,32,"27,7",18,"93,8 %","695,00 Mio. €","21,72 Mio. €",argentinien,3437
Niederlande,25,"26,7",11,"48,0 %","494,40 Mio. €","19,78 Mio. €",niederlande,3379
Uruguay,41,"26,9",14,"58,5 %","478,40 Mio. €","11,67 Mio. €",uruguay,3449


In [9]:
teilnehmer = df_wm$slug
teilnehmer

[1] "england"            "frankreich"         "brasilien"         
 [4] "deutschland"        "portugal"           "spanien"           
 [7] "argentinien"        "niederlande"        "uruguay"           
[10] "serbien"            "kroatien"           "belgien"           
[13] "senegal"            "danemark"           "polen"             
[16] "schweiz"            "marokko"            "vereinigte-staaten"
[19] "ghana"              "kanada"             "kamerun"           
[22] "wales"              "mexiko"             "ecuador"           
[25] "australien"         "tunesien"           "saudi-arabien"     
[28] "japan"              "sudkorea"           "katar"             
[31] "costa-rica"         "iran"

# Scrape Columns
We wrote a function, that takes a url to scrape all tables

In [36]:
scrapeGame = function(year, country, club_id, game_name) {

  # base url

  url = paste0("https://www.transfermarkt.de/",
               country,
               "/spielplan/verein/",
               club_id,
               "/plus/0?saison_id=", year)
    
  # print url
  print(url)
    
  # get status code
  res = httr::GET(url)

  # read response
  code = httr::status_code(res)
    
  # print status code
  print(code)
    
  # read url

  url %>%
    read_html() -> html

  # variables

  columnValues = list() # a list containing all values
  columnNames = NULL
  box = NULL # searching for the table_header

  # Obtain the correct responsive-table by name of table_header

  html %>%
    html_elements('.box') -> boxes

  # Print the table headers

  boxes

  # Iterate over each table header to find the match with table_header
  # argument

  for (item in boxes) {

    #print(item)

    if(length(html_elements(item, '.table-header') %>% html_children())!= 0) {
      item %>%
        html_elements('.table-header a img') %>%
        html_attr('title') -> name

      #print(name)
    } else {
      item %>%
        html_elements('.table-header') %>%
        html_text() %>%
        trimStringArray() -> name
        
        #print(name)
    }

    if (length(name) != 0 && grepl(tolower(game_name), tolower(name))) {
      box = item
      break
    }
  }

  box

  if(!is.null(box)) {
      # Obtain the responsive_table

      box %>%
          html_elements('.responsive-table') -> responsive_table

      # print the responsive-table

      responsive_table

      # obtain the columnNames

      responsive_table %>%
          html_elements('th') %>%
          html_text -> columnNames

      # print columnNames

      columnNames
      
      # add game_name to columnNames
      
      columnNames = append(columnNames, 'game_name')
      columnNames = append(columnNames, 'country')

      # obtain length of a all true columns

      responsive_table %>%
          html_elements('tbody tr:nth-of-type(1) td') %>%
          length() -> trueColumnsLength

      # print the length of all true columns

      trueColumnsLength

      for (i in 1:trueColumnsLength) {
          # create css selector string
          # td:not([class*="no-border-rechts"]
          string = paste0('tbody tr td:not([class*="no-border-rechts"]):nth-of-type(', i,')')

          # print the css selector string

          string

          # obtain all row elements of specified column

          responsive_table %>%
              html_elements(string) -> html_node

          # print all row elements of specified column

          html_node

          # only add the values if they are valid!!!

          if(length(html_node) != 0) {
              # obtain the children (if any) from the row elements
              # of specified column

              html_node %>%
                  html_children() -> children

              # print the children

              children

              # check the number of children contained within

              num_of_children = length(children)

              # print the number of children (if any)

              num_of_children

              # check if a child contains any subchildren

              children %>%
                  html_children() -> subchildren

              # print subchildren

              subchildren

              # check the number of children contained within

              num_of_subchildren = length(subchildren)

              # print number of subchildren

              num_of_subchildren

              # dynammically apply a search for the values
              # according to the number of children

              if (num_of_subchildren > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a > span') %>%
                      html_text() -> values
              } else if (num_of_children > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a') %>%
                      html_attr('title') -> values
              } else {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_text -> values
              }

              # wrap the obtained values within a list

              list(values)

              # iterate over all children by

              values = trimStringArray(values)

              # Add this column to the columnValues list

              columnValues = append(columnValues, list(values))
          }
          else {
              #print(paste0("Column ", i, " is not a valid value"))
          }

      }
        
      # add game_name to columnValues
      
      columnValues = append(columnValues, name)
      
      # add country name to columnValues
      columnValues = append(columnValues, country)
      
      # return data frame
      # change column names by replacing whitespace with underscore
      columnNames = gsub(" ", "_", columnNames)

      # create a dataframe with first colum and append the rest (workaround)
      df = data.frame(columnValues[[1]])

      for (i in 2:length(columnValues)) {
          df[i] = columnValues[[i]]
      }

      names(df) = columnNames
      return(df)
      
  } else {
      return(data.frame())
  }
}

# Testing Section

In [87]:
#scrapeGame(year = '2009', country = 'England', club_id = 3299, game_name = "weltmeisterschaft [0-9]+")

scrapeGameByYear = function(year, country) {
    
    df_wm %>%
        filter(slug == country) -> tempDf
    
    tempDf$club_id -> club_id
    
    #pmap_df(list('2015', 'England', '3299', c('weltmeisterschaft [0-9]+', 'freundschaftsspiele', 'em-qualifikation', 'wm-qualifikation europa', 'europameisterschaft [0-9]+')), scrapeGame)
    return(pmap_df(list(year, country, club_id,
                        c(
                            #'weltmeisterschaft [0-9]+',
                            #'freundschaftsspiele'
                            #'em-qualifikation'
                            'wm-qualifikation europa'
                            #'europameisterschaft [0-9]+',
                            #'ostasienmeisterschaft',
                            #'wm-qualifikation asien',
                            #'asienmeisterschaft [0-9]+',
                            #'asienmeisterschaft qualifikation'
                            #'copa [a-z]+ [0-9]+',
                            #'fifa arab cup',
                            #'gold cup [0-9]+'
                        )), scrapeGame))
}

In [88]:
scrapeSomething = function(country) {
    return(map2_df(seq('2009', '2021', by=1), country, scrapeGameByYear))
}

## Scrape world cup & friendlies for all 32 countries

In [89]:
complete_df = map_df(teilnehmer, scrapeSomething)

[1] "https://www.transfermarkt.de/england/spielplan/verein/3299/plus/0?saison_id=2009"
[1] 200
[1] "https://www.transfermarkt.de/england/spielplan/verein/3299/plus/0?saison_id=2010"
[1] 200
[1] "https://www.transfermarkt.de/england/spielplan/verein/3299/plus/0?saison_id=2011"
[1] 200
[1] "https://www.transfermarkt.de/england/spielplan/verein/3299/plus/0?saison_id=2012"
[1] 200
[1] "https://www.transfermarkt.de/england/spielplan/verein/3299/plus/0?saison_id=2013"
[1] 200
[1] "https://www.transfermarkt.de/england/spielplan/verein/3299/plus/0?saison_id=2014"
[1] 200
[1] "https://www.transfermarkt.de/england/spielplan/verein/3299/plus/0?saison_id=2015"
[1] 200
[1] "https://www.transfermarkt.de/england/spielplan/verein/3299/plus/0?saison_id=2016"
[1] 200
[1] "https://www.transfermarkt.de/england/spielplan/verein/3299/plus/0?saison_id=2017"
[1] 200
[1] "https://www.transfermarkt.de/england/spielplan/verein/3299/plus/0?saison_id=2018"
[1] 200
[1] "https://www.transfermarkt.de/england/spielpla

In [90]:
complete_df

_Spieltag,Datum,Uhrzeit,Ort,Rang,Gegner,Spielsystem,Zuschauer,Ergebnis,game_name,country
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Gruppe H,Fr. 07.09.2012,20:45,A,,Moldawien,4-2-3-1,10.250,0:5,WM-Qualifikation Europa,england
Gruppe H,Di. 11.09.2012,21:00,H,,Ukraine,4-2-3-1,68.102,1:1,WM-Qualifikation Europa,england
Gruppe H,Fr. 12.10.2012,21:00,H,,San Marino,4-4-2 doppel 6,84.654,5:0,WM-Qualifikation Europa,england
Gruppe H,Mi. 17.10.2012,17:00,A,,Polen,4-2-3-1,47.000,1:1,WM-Qualifikation Europa,england
Gruppe H,Fr. 22.03.2013,21:00,A,,San Marino,4-4-2,4.980,0:8,WM-Qualifikation Europa,england
Gruppe H,Di. 26.03.2013,21:00,A,,Montenegro,4-2-3-1,12.000,1:1,WM-Qualifikation Europa,england
Gruppe H,Fr. 06.09.2013,21:00,H,,Moldawien,4-2-3-1,61.607,4:0,WM-Qualifikation Europa,england
Gruppe H,Di. 10.09.2013,20:45,A,,Ukraine,4-1-4-1,66.000,0:0,WM-Qualifikation Europa,england
Gruppe H,Fr. 11.10.2013,21:00,H,,Montenegro,4-2-3-1,83.807,4:1,WM-Qualifikation Europa,england


# Split Result in Goals

In [16]:
#df_football %>%
#    mutate(goals_scored_england = ternary_v(strsplit_custom_v(Ergebnis, ':', 1), strsplit_custom_v(Ergebnis, ':', 2), (Heimmannschaft=='England'))) -> df_football_goals
#df_football_goals

## We save all data for the WorldCups&friendlies as csv

In [91]:
#complete_df
write.csv2(complete_df, file='./football_em_qualification.csv', row.names = F)

## Modelling - Data Cleaning and Transformation

In [92]:
data_set = read.csv('https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/world_cup%26friendlies_football.csv', sep=';')
fifa_ranking = read.csv("https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/FIFA_Ranking.csv", header = TRUE, sep = ",", stringsAsFactors=FALSE, fileEncoding="latin1")


In [104]:
# please Insert here

In [73]:
data_set_reduced = data_set %>%
    select(c('Ort', 'Gegner', 'Ergebnis', 'game_name', 'country'))

## Obtain goals of enemy and country

In [85]:
data_set_reduced %>%
    separate(Ergebnis, c('ergebnis_left', 'ergebnis_right')) %>%
    mutate(goals_country = ternary_v(ergebnis_left, ergebnis_right, Ort=='H')) %>%
    mutate(goals_enemy = ternary_v(ergebnis_left, ergebnis_right, Ort=='A')) -> data_set_increased

Warning message:
“Expected 2 pieces. Additional pieces discarded in 38 rows [51, 53, 212, 302, 305, 417, 471, 528, 530, 531, 583, 611, 612, 654, 687, 823, 824, 830, 831, 832, ...].”


In [86]:
data_set_increased

Ort,Gegner,ergebnis_left,ergebnis_right,game_name,country,goals_country,goals_enemy
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
H,Vereinigte Staaten,1,1,Weltmeisterschaft 2010,england,1,1
H,Algerien,0,0,Weltmeisterschaft 2010,england,0,0
A,Slowenien,0,1,Weltmeisterschaft 2010,england,1,0
A,Deutschland,4,1,Weltmeisterschaft 2010,england,1,4
H,Ägypten,3,1,Freundschaftsspiele,england,3,1
H,Mexiko,3,1,Freundschaftsspiele,england,3,1
H,Japan,2,1,Freundschaftsspiele,england,2,1
H,Ungarn,2,1,Freundschaftsspiele,england,2,1
H,Frankreich,1,2,Freundschaftsspiele,england,1,2
